In [2]:
from datasets import load_dataset, DatasetDict, Dataset
from transformers import EvalPrediction
from setfit import SetFitModel, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import train_test_split
import wandb
import pandas as pd
import torch

c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Save the path to the different datasets

In [3]:
train_en_path = "./data_sources/train/train_en.csv"
test_en_path = "./data_sources/test/test_en.csv"

train_it_path = "./data_sources/train/train_it.csv"
test_it_path = "./data_sources/test/test_it.csv"

train_es_path = "./data_sources/train/train_es.csv"
test_es_path = "./data_sources/test/test_es.csv"

### Set up W&B

In [4]:
wandb.login()

wandb: Currently logged in as: sravisconti (sravisconti-projects) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

### Load data in DatasetDict

In [13]:
# Load CSV manually for the train split
train_df = pd.read_csv(train_it_path)
test_df = pd.read_csv(test_it_path)

# Convert back to Hugging Face Datasets
dataset_it = DatasetDict({
    "train": Dataset.from_pandas(train_df.reset_index(drop=True)),
    "test": Dataset.from_pandas(test_df)
})

### Define Metrics

In [11]:
def compute_metrics(preds, labels):
    precision_macro = precision_score(labels, preds, average="macro", zero_division=0)
    recall_macro = recall_score(labels, preds, average="macro", zero_division=0)
    f1_macro = f1_score(labels, preds, average="macro", zero_division=0)

    return {
        "precision_macro": precision_macro,
        "recall_macro": recall_macro,
        "macro_f1": f1_macro,
    }


### Train and Evaluate

In [15]:
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SetFitModel.from_pretrained(model_name)

# to match labels with meaning: 0 --> "offensive", 1 --> "reappropriative"
model.labels = ["offensive", "reappropriative"]

args = TrainingArguments(
    batch_size=8,
    num_epochs=2,
    num_iterations=5,
    report_to="wandb"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset_it["train"],
    metric=compute_metrics,
    column_mapping={"text": "text", "label": "label"}
)

wandb.init(project="multi-pride-setfit-pipeline", name="setfit-it")

trainer.train()
final_results = trainer.evaluate(dataset_it["test"])
wandb.log(final_results)
print("Final test metrics:", final_results)

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.
Applying column mapping to the training dataset
Map: 100%|██████████| 868/868 [00:00<00:00, 11473.43 examples/s]


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


macro_f1,▁
precision_macro,▁
recall_macro,▁
train/embedding_loss,▆▅▅▄▃▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▃▂▁▁▁█▄▂▁▁▁▇
train/epoch,▁▁▁▁▂▂▂▂▂▂▂▂▂▂▃▃▃▃▃▃▃▃▃▃▃▄▁▁▂▂▃▄▅▆█▁▂▅▆▁
train/global_step,▁▁▁▂▂▂▃▃▃▄▄▄▄▅▅▆▆▆▆▆▆▇▇▇▇██▁▁▁▂▂▂▂▃▂▂▂▂▃
train/grad_norm,▄▃▃▁▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▂▁▁▁▁▁▆▆▂▁▁▁▁█
train/learning_rate,▁▁▁▁▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▆▇▇▇▇▇▇█▇▆▆▄▃▃▃▂▅█▄▃▁
macro_f1,0.87399
precision_macro,0.85735
recall_macro,0.89448


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\wandb\analytics\sentry.py:263: DeprecationWarning: The `Scope.user` setter is deprecated in favor of `Scope.set_user()`.
  self.scope.user = {"email": email}


***** Running training *****
  Num unique pairs = 8680
  Batch size = 8
  Num epochs = 2
c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
1,0.280100
50,0.178600
100,0.137500
150,0.111500
200,0.098400
250,0.067200
300,0.060600
350,0.043700
400,0.019500
450,0.006800


c:\Users\sravi\Desktop\Projects\multi-pride\.venv\Lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
Applying column mapping to the evaluation dataset
***** Running evaluation *****


Final test metrics: {'precision_macro': 0.8738624873609707, 'recall_macro': 0.9001623376623377, 'macro_f1': 0.8861024033437827}
